# Master TIDE - Conférences Python 2021

### Natural Language Processing

Francis Wolinski

&copy; 2021 Yotta Conseil

In [ ]:
# imports
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#no warning
import warnings
warnings.simplefilter("ignore")

pd.set_option('display.max_rows', 30)
pd.set_option('display.max_colwidth', 100)

## 1. Rappel sur les chaînes de caractères

En Python, les chaînes de caractères sont des instances de la classe `str`. Ce sont des collections ordonnées immuables de caractères.

In [ ]:
# exemple
s = 'Un exemple de chaîne de caractères'
type(s)

Une chaîne de caractères doit être vue comme une collection d'octets (bytes) qui représente la chaîne en machine et d'un encodage qui exprime la manière dont les caractères sont codés.

La méthode `encode()` permet d'encoder une chaîne en octets selon un encodage particulier, par défaut en `UTF-8`. On obtient un objet de type `bytes`.

Python gère tout un ensemble d'encodages: `utf-8` (par défaut en Python 3), `ascii`, `latin-1` ou `ISO-8859-1`, `ISO-8859-15`, `cp1252`, etc.

In [ ]:
# encodage en UTF-8
s.encode()

In [ ]:
# encodage en latin-1 ou ISO-8859-1
s.encode('latin-1')

In [ ]:
# decodage bytes --> str
b'Un exemple de cha\xeene de caract\xe8res'.decode('latin-1')

In [ ]:
# exemple d'écriture de fichier en UTF-8
with open('texte.txt', 'w', encoding='utf-8') as f:
    f.write(s + '\n' + s)

In [ ]:
# exemple de lecture de fichier en UTF-8
with open('texte.txt', encoding='utf-8') as f:
    texte = f.read()
texte

In [ ]:
# exemple de lecture de fichier en latin-1
with open('texte.txt', encoding='latin-1') as f:
    texte = f.read()
texte

## 2. Traitement vectoriel avec `pandas`

La librairie `pandas` permet d'effectuer un traitement vectoriel des textes en utilisant l'accesseur `str`.

Il existe de nombreuses méthodes de la classe `str` reprises dans `pandas`. Par exemple :
- `startswith()` : teste si une chaîne commence par une sous-chaîne
- `endswith()` : teste si une chaîne se termine par une sous-chaîne
- `contains()` : teste si une chaîne contient un motif (regex)
- `extract()` : extrait un ou plusieurs motifs (regex)

In [ ]:
# chargement d'un fichier texte dans une Series
df = pd.read_csv('imdb_master.csv', usecols=['review'], encoding='latin-1')
reviews = df['review']
reviews.head()

In [ ]:
# sélection
reviews.loc[reviews.str.startswith('Very ')]

In [ ]:
# sélection
reviews.loc[reviews.str.contains('Hitchcock')]

In [ ]:
# sélection
reviews.loc[reviews.str.contains('Hitchcock', flags=re.I)]

In [ ]:
# extract
tab = reviews.str.extract(r'(\w+) Hitchcock', expand=False)
var = tab.dropna()
var

In [ ]:
# mots précèdent Hitchcock
var.value_counts()

In [ ]:
# alignement de textes
def search(word):
    selection = reviews[reviews.str.contains(word)]
    result = selection.apply(lambda s: (' '*25 + s + ' '*25)[s.find(word):s.find(word) + 50 + len(word)])    
    return result

In [ ]:
search('Hitchcock')

In [ ]:
search('Ã')

In [ ]:
# cleaning

# replace character encoding mistakes
reviews = reviews.apply(lambda x: x.replace('Ã¡', 'á'))
reviews = reviews.apply(lambda x: x.replace('Ã ', 'à'))
reviews = reviews.apply(lambda x: x.replace('Ã ', 'à'))
reviews = reviews.apply(lambda x: x.replace('Ã¢', 'â'))
reviews = reviews.apply(lambda x: x.replace('Ã\xa0', 'à'))
reviews = reviews.apply(lambda x: x.replace('Ã¥', 'å'))
reviews = reviews.apply(lambda x: x.replace('Ã£', 'ã'))
reviews = reviews.apply(lambda x: x.replace('Ã»', 'â'))
reviews = reviews.apply(lambda x: x.replace('Ã§', 'ç'))
reviews = reviews.apply(lambda x: x.replace('Ã©', 'é'))
reviews = reviews.apply(lambda x: x.replace('Ã¨', 'è'))
reviews = reviews.apply(lambda x: x.replace('Ã«', 'ë'))
reviews = reviews.apply(lambda x: x.replace('Ãª', 'ê'))
reviews = reviews.apply(lambda x: x.replace('Ã¯', 'ï'))
reviews = reviews.apply(lambda x: x.replace('Ã®', 'î'))
reviews = reviews.apply(lambda x: x.replace('Ã¬', 'ì'))
reviews = reviews.apply(lambda x: x.replace('Ã\xad', 'í'))
reviews = reviews.apply(lambda x: x.replace('Ã±', 'ñ'))
reviews = reviews.apply(lambda x: x.replace('Ã³', 'ó'))
reviews = reviews.apply(lambda x: x.replace('Ã²', 'ò'))
reviews = reviews.apply(lambda x: x.replace('Ã¶', 'ö'))
reviews = reviews.apply(lambda x: x.replace('Ã´', 'ô'))
reviews = reviews.apply(lambda x: x.replace('Ãµ', 'õ'))
reviews = reviews.apply(lambda x: x.replace('Ã°', 'ð'))
reviews = reviews.apply(lambda x: x.replace('Ã¸', 'ø'))
reviews = reviews.apply(lambda x: x.replace('Ãº', 'ú'))
reviews = reviews.apply(lambda x: x.replace('Ã¹', 'ù'))
reviews = reviews.apply(lambda x: x.replace('Ã¼', 'ü'))
reviews = reviews.apply(lambda x: x.replace('Ã½', 'ý'))
reviews = reviews.apply(lambda x: x.replace('Ã¿', 'ÿ'))
reviews = reviews.apply(lambda x: x.replace('Ã?', 'Æ'))
reviews = reviews.apply(lambda x: x.replace('Ã¦', 'æ'))
# br
reviews = reviews.apply(lambda x: x.replace('<br />', ' '))

## 3. Modèle du sac de mots (bag-of-words)

Le modèle du sac de mots est un modèle dans lequel un texte est représenté par un dictionnaire (non ordonné) fréquentiel des mots : nombre d'occurences du mot dans un texte, nombre de documents contenant ce mot pour un corpus.

Il est possible au préalable de défléchir les mots, c'est-à-dire de les remplacer par une forme simplifiée : passage en minuscules, lemmatisation (masculin, singulier, infinitif), racinisation (noyau lexical).

In [ ]:
# import
from collections import Counter

In [ ]:
'A bon chat bon rat'.split()

In [ ]:
# exemple
c = Counter()
c.update('A bon chat bon rat'.split())
c

In [ ]:
# texte
reviews[0]

In [ ]:
# calcul du Counter
pattern = r'[A-Za-zÀ-ÿ0-9]+'
c = Counter()
c.update(re.findall(pattern, reviews[0]))
c

In [ ]:
# mise en Series
s = pd.Series(c)
s = s.sort_values(ascending=False)
s

In [ ]:
# tous les textes
pattern = r'[A-Za-zÀ-ÿ0-9]+'
c = Counter()
reviews.apply(lambda x: c.update(re.findall(pattern, x)))
vocab = pd.Series(c)
vocab = vocab.sort_values(ascending=False)
vocab

In [ ]:
# dictionaire fréquentiel
# tous les textes
pattern = r'[A-Za-zÀ-ÿ0-9]+'
c = Counter()
reviews.apply(lambda x: c.update(set(re.findall(pattern, x))))
vocab = pd.Series(c)
vocab = vocab.sort_values(ascending=False)
vocab

In [ ]:
# stop words
from nltk.corpus import stopwords
stopwords_en = stopwords.words("english")

stopwords_en

In [ ]:
# élisions en anglais
[w for w in stopwords_en if "'" in w]

In [ ]:
# dictionaire fréquentiel
# tous les textes
# mots en minuscules
# sans stop words
pattern = r'[A-Za-zÀ-ÿ0-9]+(?:\'(?:d|ll|re|s|t|ve))?'
c = Counter()
reviews.apply(lambda x: c.update(set(re.findall(pattern, x.lower()))))
vocab = pd.Series(c)
vocab = vocab.drop(stopwords_en, errors='ignore')
vocab = vocab.sort_values(ascending=False)
vocab

## 4. Calcul de bi-grammes

Recherche de cooccurrence de mots.

In [ ]:
import nltk
from nltk import word_tokenize

from tqdm import tqdm

def get_doc():
    for text in tqdm(reviews[:10_000]):
        words = word_tokenize(text.lower())
        doc = [word for word in words if not word in stopwords_en]
        yield doc

bgm = nltk.collocations.BigramAssocMeasures()
finder = nltk.collocations.BigramCollocationFinder.from_documents(get_doc())
finder.apply_freq_filter(3)
scored = finder.score_ngrams(bgm.likelihood_ratio)

# save to pickle
import pickle
with open('scored.pkl', 'wb') as f:
    pickle.dump(scored, f)

In [ ]:
# load from pickle
import pickle
with open('scored.pkl', 'rb') as f:
    scored = pickle.load(f)

In [ ]:
# bi-grammes trouvés
scored

In [ ]:
# nb de bi-grammes
len(scored)

In [ ]:
def search(word1=None, word2=None, n=10):
    i = 0
    for (w1, w2), s in scored:
        if (word1 is None or (w1.find(word1)!=-1)) \
            and (word2 is None or (w2.find(word2)!=-1)):
            print(w1, w2, s)
            i += 1
            if i == n:
                 break

In [ ]:
search('john')

## 5. Vector space model avec scikit-learn TF-IDF

En recherche d'information, `TF–IDF`, abréviation de **term frequency–inverse document frequency**, est un indicateur statistique qui cherche à capturer l'importance d'un terme dans un document au sein d'un corpus.

Un corpus contient `N` documents. Un terme apparaît dans `n` documents du corpus. pour un document donné de longueur `L`, un terme apparaît `T` fois.

`TF–IDF` est le produit de 2 facteurs :
- *term frequency* (dans un document donné) :
    - $\frac{T}{L}$
- *inverse document frequency* (fréquence d'apparition dans le corpus):
    - $\log(\frac{N}{n})$
    
Dans ce modèle, chaque document est représenté par un vecteur normalisé composé du calcul `TF–IDF` pour chaque terme qu'il contient (vector space model).

La librairie `scikit-learn` permet de calculer les vecteurs `TF–IDF` pour un corpus de documents.

Cette librairie comprend différentes classes traitant des textes:

- `CountVectorizer`: implémente la tokenisation et le décompte des occurrences
- `TfidfTransformer`: transforme une matrice de comptage en une représentation `TF` ou `TF–IDF`
- `TfidfVectorizer`: équivalent à *CountVectorizer* suivi par *TfidfTransformer*

Ces classes produisent des **matrices creuses**, c'est-à-dire des matrices pleines de 0 à 99% ou 99.9%, puisque chaque document ne contient que *100* ou *1000* termes et que l'ensemble d'un corpus peut contenir plusieurs *100K* termes.

*Nota bene*

- Traiter des matrices creuses nécessite une certaine attention car il est générallement pas possible de les transformer en *numpy* arrays ou *pandas* dataframes, à cause des limitations de mémoire.

<div class="alert alert-warning">
<b>Pour plus d'information</b>
<ul>
    <li><a href='https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction'>https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction</a></li>
</ul>
</div>

### 5.1 CountVectorizer

`CountVectorizer`: implémente la tokenisation des termes et le comptage des occurrences.

Il effectue le même calcul que `Counter`. On peut calculer facilement le nombre de fois que chaque terme apparaît dans un document, ou dans tout le corpus.

In [ ]:
# import
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
%%time
# CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(reviews)
X

On obtient une matrice creuse de *100.000* x *144.342*, c'est-à-dire, 144 milliards de nombres, mais qui ne contient que *13.683.613* éléments:

En fait, on a :
- 100.000: documents
- 144.342: termes
- 13.683.613: éléments

In [ ]:
# densité & sparsité
density = X.count_nonzero() / X.shape[0] / X.shape[1]
print('density:', density)
sparsity = 1.0 - density
print('sparsity: {:.3f}%'.format(sparsity))

In [ ]:
# sparse matrix
type(X)

In [ ]:
# memory error!
X.toarray()

Il est possible d'utiliser la méthode `get_feature_names()` de `CountVectorizer` qui donne la liste de tous les termes du corpus, dans le même ordre que la matrice creuse.

In [ ]:
# termes d'un document donné
big_review_id = reviews.str.len().idxmax()

# ligne de la matrice avec les termes
values = X[big_review_id].toarray()[0]

# liste de tous les termes
words = vectorizer.get_feature_names()

# mise en Series
vocab = pd.Series(values, index=words)

# nombre total de termes et nombre de termes dans le document
print(len(vocab), len(vocab[vocab!=0]))
vocab = vocab[vocab!=0]
vocab = vocab.sort_values(ascending=False)
vocab.head(30)

In [ ]:
# termes de l'ensemble des documents en sommant selon l'axis 0
# somme en colonne par terme
values = np.asarray(X.sum(axis=0))[0]

# liste de tous les termes
words = vectorizer.get_feature_names()

# mise en Series
vocab = pd.Series(values, index=words)
vocab = vocab.sort_values(ascending=False)
vocab.head(30)

### 5.2 TfidfVectorizer

In [ ]:
# import
from sklearn.feature_extraction.text import TfidfVectorizer

Afin d'éliminer les stop words, les mots rares ou inconnus, on utilise le `TfidfVectorizer` avec des paramètres particuliers. Le résultat est une matrice creuse plus petite.

In [ ]:
TfidfVectorizer?

In [ ]:
%%time
# TfidfVectorizer
pattern = '[A-Za-zÀ-ÿ0-9]+(?:\'(?:d|ll|re|s|t|ve))?'
vectorizer = TfidfVectorizer(analyzer='word',
                             token_pattern=pattern,
                             min_df=10,
                             stop_words=stopwords_en)
X = vectorizer.fit_transform(reviews)
X

In [ ]:
# density & sparsity
density = X.count_nonzero()/X.shape[0]/X.shape[1]
print('density:', density)
sparsity = 1.0 - density
print('sparsity: {:.3f}%'.format(sparsity))

Il est possible de calculer la somme des `TD-IDF` de chaque terme pour l'ensemble du corpus.

In [ ]:
# termes de tous les documents en sommant selon l'axis 0
# somme en colonnes
values = np.asarray(X.sum(axis=0))[0]
index = vectorizer.get_feature_names()
vocab = pd.Series(values, index=index)
vocab = vocab.sort_values(ascending=False)
vocab.head(30)

Nous allons utiliser la vectorisation des documents avec `TF-IDF` pour rechercher les documents similaires à un document donné.

In [ ]:
# sélection d'un document dans le corpus
review_id = 13473
df.loc[review_id, 'review']

In [ ]:
# indices des termes présents dans ce document
indices = np.nonzero(X[review_id].toarray())[1]
indices

In [ ]:
# termes correspondants
termes = [vectorizer.get_feature_names()[i] for i in indices]
termes

In [ ]:
# valeur TF-IDF de chaque terme
values = [X[review_id, i] for i in indices]
values

In [ ]:
# mise en Series
s = pd.Series(values, index=termes)
s = s.sort_values(ascending=False)
s

Pour trouver les documents similaires, nous calculons le produit scalaire entre le vecteur de chacun des documents et le vecteur du document sélectionné, on conserve les documents qui produisent les meilleurs scores.

In [ ]:
# produit scalaire du document donné avec lui-même
np.dot(X[review_id], X[review_id].T).toarray()[0]

In [ ]:
# indices des 10 meilleurs documents
ids = np.dot(X, X[review_id].T).toarray().T[0].argsort()[-10:]
ids

In [ ]:
# textes asociés
df.loc[ids]

Il est possible de tout mettre dans une seule fonction qui produit les n meilleurs documents.

In [ ]:
# fonction qui trouve les revues similaires
def find_similar_reviews(review_id, n=5):
    # print review
    print(reviews.loc[review_id])
    print()
    # ids des termes de la review
    word_ids = np.nonzero(X[review_id].toarray())[1]
    # termes de la review
    words = [vectorizer.get_feature_names()[word_id] for word_id in word_ids]
    # TF-IDF des termes de la review
    values = [X[review_id, word_id] for word_id in word_ids]
    # mises en Series et print
    s = pd.Series(values, index=words)
    s = s.nlargest(20)
    print(s)
    print()
    # calcul du produit matriciel entre la matrice et la revue
    # on retient du n° 2 au n° n+1
    ids = np.dot(X, X[review_id].T).toarray().T[0].argsort()[-n-1:-1]
    # collecte des reviews et print
    results = df.loc[ids[::-1], 'review']
    for i in range(len(results)):
        print(i+1)
        print(results.iloc[i])
        print()

In [ ]:
# test
find_similar_reviews(review_id)

In [ ]:
# random review
np.random.seed(1)
review_id = np.random.randint(100_000)
find_similar_reviews(review_id)

Les autres applications de `TF-IDF` sont la classification, le clustering et l'extraction de thématiques de documents.

## 6 Gensim

**Gensim** : topic modelling for humans

### 6.1 Word2Vec

Hypothèses :
- un corpus composé de documents
- les documents sont composés de mots

L'algorithme utilise le plongement lexical (word embeddings) (i.e. le contexte local) pour produire des vecteurs pour chacun des mots :

- préprocessing des documents : `gensim.utils.simple_preprocess()`
- modélisation des mots : `gensim.models.Word2Vec()`

Ensuite, il est possible d'utiliser les propriétés algébriques des vecteurs pour calculer et utiliser la similarité entre des mots. L'utilisation typique de `gensim` est d'aider à trouver des synonymes ou des antonymes pour des applications dédiées de text mining.

- `most_similar(positive=[...])`: trouve les entités similaires à une liste d'entités (synonymes)
- `most_similar(positive=[...], negative=[...])`: trouve les entités similaires à une liste d'entités (synonyms) et celles qui sont le moins similaires à une autre (antonymes)
- `closer_than(entity1, entity2)`:  trouve les entités plus proches de `entity1` que `entity2` l'est de `entity1`
- `doesnt_match()`: trouve l'entité qui correspond le moins au autres

Il y a beaucoup d'autres techniques du type "X2Vec" : e.g., Doc2Vec, Fact2Vec.

Installation, ouvrir un terminal dans le dossier `anaconda3/condabin` :

- PC: `.\conda install -c anaconda gensim`
- Mac: `./conda install -c anaconda gensim`

<div class="alert alert-warning">
<b>Further reading</b>
<ul>
    <li><a href='https://radimrehurek.com/gensim/index.html'>https://radimrehurek.com/gensim/index.html</a></li>
</ul>
</div>

In [ ]:
# import

import gensim

#### Dataset

#### Application de l'algorithme de `gensim `

In [ ]:
%%time
# preprocesing & modelling
if False:
    # step 1: preprocessing texts
    documents = reviews.apply(lambda x: gensim.utils.simple_preprocess(x)).values
    # step 2: process documents
    # time depends on vector size, here 150
    model = gensim.models.Word2Vec(documents,
                              size=150,
                              window=10,
                              min_count=2,
                              workers=4)
    model.save('imdb_master.bin')
else:
    model = gensim.models.Word2Vec.load('imdb_master.bin')

#### Utilisation du modèle

La fonction `most_similar()` prend des mots positifis et/ou negatifs, afin de trouver des mots similaires, ou non similaires.

C'est utile pour fabriquer des liste de synonymes ou d'antonymes pour une application de text mining.

**Synonymes**

In [ ]:
# good
model.wv.most_similar(positive='good')

In [ ]:
# great
model.wv.most_similar(positive='great')

In [ ]:
# bad
model.wv.most_similar(positive='bad')

In [ ]:
# horrible
model.wv.most_similar(positive='horrible')

In [ ]:
# superhero
model.wv.most_similar(positive=['superhero'])

In [ ]:
# hitchcock
model.wv.most_similar(positive=['hitchcock'])

In [ ]:
# hitchcock & french
model.wv.most_similar(positive=['hitchcock', 'french'])

In [ ]:
gensim.models.Word2Vec?

**Arithmétique des mots**

In [ ]:
# +woman +king -man
model.wv.most_similar(positive=['king', 'woman'], negative=['man'])

In [ ]:
# +man +queen -woman
model.wv.most_similar(positive=['queen', 'man'], negative=['woman'])

In [ ]:
model.wv.most_similar(positive=['actor', 'woman'], negative=['man'])

**Trouver l'intrus**

In [ ]:
# doesnt_match
model.wv.doesnt_match(['thriller', 'drama', 'comedy', 'melodrama', 'romance'])

### 6.2 Topic Modelling

Le topic modelling est un outil de machine learning non supervisé.

Il permet de trouver les thèmes majeurs dans un corpus de textes, de façon automatique (ou presque).
La librairie de référence est **gensim**.

Il existe plusieurs modèle de Topic Modelling, les plus connus sont :
- LSA pour **Latent Semantic Analysis**, qui fonctionne un peu comme une analyse en composantes
principales
- LDA pour **Latent Dirichlet Allocation**, qui fonctionne un peu comme un K-means

Nous allons ici nous focaliser sur la mise en oeuvre de la LDA, mais la méthodologie est la même pour la
LSA :
- Preprocessing des données textuelles : calcul du BOW et/ou du TF-IDF
- Entrainement du modèle (e.g. LDA ou LSA)
- Interprétation des résultats et définition des thèmes majeurs
- Eventuelles itérations pour affiner les résultats

Nous allons ici nous focaliser sur la mise en oeuvre de la LDA, mais la méthodologie est la même pour la
LSA :
- Preprocessing des données textuelles : calcul du BOW et/ou du TF-IDF
- Entrainement du modèle (e.g. LDA ou LSA)
- Interprétation des résultats et définition des thèmes majeurs
- Eventuelles itérations pour affiner les résultats

#### 6.2.1 Preprocessing

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stopwords_en = stopwords.words('english')

def clean_data(text):
    text = text.lower()
    tokens = word_tokenize(text)
    tokens = [t for t in tokens if t.isalpha() and t not in stopwords_en]
    return tokens

# On effectue le traitement en tokens pour la suite
tokens = reviews[:10_000].apply(clean_data)
tokens.head()

In [ ]:
# On calcule de TF-IDF
vectorizer = TfidfVectorizer(stop_words=stopwords_en, analyzer=lambda x: x)
tfidf = vectorizer.fit_transform(tokens)
tfidf

####  6.2.2 Topic Modelling
Nous allons ensuite passer à l'étape de Topic Modelling à l'aide de gensim.

La première étape est toujours la même : instancier le modèle de LDA qu'il faut importer au préalable de la façon suivante :

```python
    from gensim.models import LdaModel
```

La classe `LdaModel` possède la signature suivante (extrait) :

```python
    lda_model = gensim.models.ldamodel.LdaModel(corpus,
                                            id2word,
                                            num_topics,
                                            random_state,
                                            chunksize,
                                            passes)
```

Avec :
- `Corpus` le TF-IDF ou le BOW (calculé avec scikit-learn)
- `id2word` un dictionnaire avec les correspondances entre indices dans le corpus et les mots (calculé par gensim)
- `num_topics` est le nombre désiré de thèmes
- `random_state` pour la reproductibilité (mettre toujours la même valeur)
- `chunksize` la taille du mini-batch (laisser par défaut)
- `passes` le nombre de fois que le modèle verra tout le corpus pour s'entraîner (1 par défaut, en général 5 ou 10 est très bien)

La seule information qui manque est le paramètre `id2word`.

On peut utiliser pour ça l'objet `Dictionary` du module `gensim.corpora`, qui s'importe de la façon suivante :
```python\n",
    from gensim.corpora import Dictionary
```
Et qui s'applique directement sur les tokens (d'où la nécessité de le calculer au-dessus) :

In [ ]:
from gensim.corpora import Dictionary
from nltk.tokenize import word_tokenize
id2word = Dictionary(tokens)

Un dernier détail avant de pouvoir utiliser le modèle, il faut convertir le TF-IDF de `scikit-learn` en un format propre à `gensim` à l'aide de la classe `Sparse2Corpus` :

In [ ]:
from gensim.matutils import Sparse2Corpus
tfidf_gensim = Sparse2Corpus(tfidf, documents_columns=False)

Finalement, il est possible d'instancier et d'entrainer notre modèle (le tout se faisant en même temps) :

In [ ]:
%%time
from gensim.models import LdaModel

# On instancie et entraine un modèle qui trouvera 3 thèmes
lda = LdaModel(corpus=tfidf_gensim,
    id2word=id2word,
    num_topics=3,
    random_state=0,
    passes=5)

Il est maintenant possible d'afficher les thèmes majeurs, avec la méthode `print_topics()` :

In [ ]:
from pprint import pprint
pprint(lda.print_topics())

L'idée est ensuite d'interpréter ces résultats...

Une possibilité est d'utiliser les vecteurs obtenus par Word2Vec pour calculer un vecteur pour chaque topic.

In [ ]:
# obtention du topic n° 2
topics = lda.show_topics(num_topics=10, num_words=10, log=False, formatted=False)
topics[2]

In [ ]:
# calcul du vecteur pondéré et des mots similaires
vector = np.zeros(150)

for word, weight in topics[2][1]:
    if word in model.wv:
        vector += model.wv[word] * weight

model.wv.most_similar(positive=[vector], topn=3)

#### 6.2.3 Visualisation des résultats

Il existe un package dédié de visualisation des résultats de LDA, nommé pyLDAvis . Il s'agit du portage en Python du package R `LDAvis`.

Il doit au préalable être installé avec la commande suivante :
```
conda install -c conda-forge pyldavis
```

L'utilisation du package est ensuite relativement simple, il suffit ensuite d'effectuer quelques imports et d'écrire quelques lignes de code :

In [ ]:
import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
bow = [id2word.doc2bow(line) for line in tokens] # convert corpus to BoW format
vis = pyLDAvis.gensim.prepare(topic_model=lda, dictionary=id2word, corpus=bow)
vis

Ce package fournit une représentation graphique :
- à gauche des topics détectés par le modèle LDA, avec leur proximité et le volume de leur
vocabulaire,
- à droite des termes du corpus et des différents topics, avec leur importance relative dans le topic.

Il est possible de sélectionner :
- un topic à gauche et de visualiser les termes correspondants,
- un terme à droite et de visualiser les topics correspondants.

## 7. WordCloud

Installation, ouvrir un terminal dans le dossier `anaconda3/condabin` :

- sur PC : `.\conda install -c conda-forge wordcloud`
- sur Mac : `./conda install -c conda-forge wordcloud`

Wordcloud est une librairie Python qui permet de fabriquer des nuages de mots. La taille des mots dépend de leur nombre d'occurences.

In [ ]:
# import et utilisation de wordcloud
from wordcloud import WordCloud

# un texte
text = reviews.loc[reviews.str.len().idxmax()]

# texte aléatoire
#from numpy import random
#random.seed(100)
#text = df.sample(1)['review'].values[0]

# à essayer avec différentes options, e.g. stopwords=[]
wc = WordCloud(max_words=100,
               stopwords=stopwords_en,
               normalize_plurals=True,
               width=800,
               height=600)
wc.generate(text)

# affichage du nuage de mots
plt.figure(figsize=(12,9))
plt.axis("off")
plt.imshow(wc);